In [ ]:
%load_ext autoreload
%autoreload 2
from rpy2 import robjects
from rpy2.robjects import r as R
from rpy2.robjects.packages import importr
import pandas as pd
R('options(repos=structure(c(CRAN="45")))')

In [ ]:
R["setwd"]("GSECA")

In [ ]:
R["source"]("Scripts/config.R")

# Cook Data

In [ ]:
directory = "../../topics/datasets/gtex10/"
Lev=2

In [ ]:
df = pd.read_csv(f'{directory}/topsbm/topsbm_level_{Lev}_topic-dist.csv', index_col=1).drop("i_doc", axis=1)
df=df.transpose().reset_index()
df.rename({"index":"id"}, axis=1, inplace=True)
df = df.set_index("id").transpose()
#df = df.subtract(df.min(axis=0),1).abs().divide(df.max(axis=0)-df.min(axis=0),1) ## threshold should be 0.005
df = df.subtract(df.mean(axis=0),1).abs().divide(df.std(0),1)
df.head(2)

In [ ]:
df_files = pd.read_csv(f'{directory}/files.dat', index_col=0).reindex(index=df.index)
df["tissue"] = df_files["SMTS"]

In [ ]:
data_df=df.groupby("tissue").mean().applymap(lambda p: p > 1)

# Prepare data

In [ ]:
def is_case(topic:str)->bool:
    return data_df.loc["Heart"][topic.replace("topic", "Topic").replace("."," ")]

In [ ]:
pd.read_csv(f"{directory}/topsbm/topsbm_level_{Lev}_word-dist.csv").rename(columns={"Unnamed: 0":"ensembl_gene_id"}).set_index("ensembl_gene_id").to_csv("../M.tsv", sep="\t")
M = R['read.delim']("../M.tsv")

In [ ]:
P_py = ["CASE" if is_case(topic) else "CNTR" for topic in R['names'](M)[1:]]
L = robjects.StrVector(P_py)
#R['names'](M)[1:]
pd.DataFrame(L).to_csv("../L.tsv", sep="\t", header=True, index=False)

# Run

In [ ]:
devtools = importr("devtools")

# Postprocessing

In [ ]:
import os

In [ ]:
res_dir = os.listdir("../Results")
res_dir

In [ ]:
pd.read_csv(f"../Results/{res_dir[-1]}/gseca.csv").sort_values(["pv"], ascending=True)